In [1]:
# Importamos librerías de análisis de datos
%matplotlib inline
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('mode.chained_assignment', None) # Deshabilita SettingWithCopyWarning. Ojo.

In [2]:
# Importamos utilidades y modelos de sklearn
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [3]:
df = pd.read_csv('../input/prices/train.csv', parse_dates=['fecha'])
df_test = pd.read_csv('../input/prices/test.csv')
df

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,nan,2.00,1.00,2.00,80.00,80.00,23533.00,nan,nan,2015-08-23,0.00,0.00,0.00,0.00,0.00,2273000.00
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.00,3.00,2.00,2.00,268.00,180.00,24514.00,19.31,-99.23,2013-06-28,0.00,0.00,0.00,1.00,1.00,3600000.00
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.00,3.00,2.00,2.00,144.00,166.00,48551.00,nan,nan,2015-10-17,0.00,0.00,0.00,0.00,0.00,1200000.00
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.00,2.00,1.00,1.00,63.00,67.00,53666.00,19.30,-99.69,2012-03-09,0.00,0.00,0.00,1.00,1.00,650000.00
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.00,2.00,1.00,1.00,95.00,95.00,47835.00,nan,nan,2016-06-07,0.00,0.00,0.00,0.00,0.00,1150000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,119879,bonita casas de 2 recamaras a 10 minutos del c...,vendo casa en bosques de ica residencial a 10 ...,Casa,BOSQUES,Zinacantepec,Edo. de México,0.00,2.00,2.00,1.00,67.00,nan,53666.00,nan,nan,2015-02-08,0.00,0.00,0.00,0.00,0.00,650000.00
239996,259178,casa en condominio a 10 min. del centro de toluca,"casa con un jardin amplio, un cuarto de servic...",Casa,Filiberto Navas 325,Toluca,Edo. de México,0.00,3.00,3.00,3.00,200.00,250.00,51954.00,19.29,-99.69,2014-07-10,0.00,0.00,0.00,1.00,1.00,1940000.00
239997,131932,nicolas san juan,"departamento con excelente ubicación, muy cerc...",Apartamento,Nicolas San Juan,Benito Juárez,Distrito Federal,20.00,2.00,1.00,2.00,138.00,138.00,50003995.00,nan,nan,2015-03-03,0.00,0.00,0.00,0.00,0.00,3400000.00
239998,146867,casa sola. javier rojo gomez.,"casa sola, dividida en cuatro departamentos de...",Casa,Javier Rojo Gomez 120,Iztapalapa,Distrito Federal,20.00,4.00,0.00,4.00,235.00,137.00,24162.00,19.37,-99.08,2014-12-26,1.00,0.00,0.00,1.00,1.00,2890000.00


**FEATURE ENGINEERING**

In [4]:
def asignarMetros(metroscubiertos, metrostotales):
    if (metroscubiertos != 0 and metrostotales == 0 or metroscubiertos > metrostotales):
        return metroscubiertos
    else:
        return metrostotales

In [5]:
def agregarAnioMesYPrecioDelDolarALaFecha(dfReceived):
    dfReceived['fecha'] = pd.to_datetime(dfReceived['fecha'])
    dfReceived['year'] = dfReceived['fecha'].dt.year
    dfReceived['month']= dfReceived['fecha'].dt.month
    dfReceived= pd.merge(dfReceived, dolar, on=['year', 'month'], how='left')
    dfReceived.rename(columns={"price": "precioDelDolar"}, inplace = True)
    return dfReceived

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns

dolar= pd.read_csv('../input/prices/Datos histricos USD_MXN.csv')
dolar.columns = ["fecha", "last", "opening", "max", "min", "std"]
dolar.drop(["std", "opening", "opening", "max", "min"], axis=1 ,inplace=True)
#Paso a formato fecha correctamente y luego creo columnas mes y año
dolar['date'] = pd.to_datetime(dolar['fecha'], format="%b %Y")
dolar.drop(["fecha"], axis=1, inplace=True)
dolar.columns= ['price', "date"]
#El csv ponia los numeros con punto en vez de coma.
dolar['price'] = dolar['price'].replace(',', '.', regex=True).astype(float)
dolar["year"] = dolar.date.dt.year
dolar['month'] = dolar.date.dt.month
dolar.drop(["date"], axis=1, inplace=True)

In [7]:
propiedades = df.groupby(['tipodepropiedad'])
propiedades = propiedades.agg({
         'habitaciones': "mean",    # Sum duration per group
         'banos': "mean",  # get the count of networks
         'garages': 'mean'  # get the first date per group
    })

propiedades["habitaciones"].fillna(0, inplace = True)
propiedades["banos"].fillna(0, inplace = True)

propiedades

,habitaciones,banos,garages
tipodepropiedad,,,
Apartamento,2.39,1.82,1.41
Bodega comercial,2.63,1.85,0.48
Casa,3.09,2.24,1.71
Casa en condominio,2.96,2.37,1.91
Casa uso de suelo,4.28,2.40,1.43
Departamento Compartido,2.60,1.79,1.40
Duplex,2.54,1.36,1.18
Edificio,6.13,2.63,0.73
Garage,0.00,0.00,0.00


In [8]:
propiedades.loc["Casa"]["habitaciones"]

3.0920804457323725

In [9]:
df.iloc[17512]

id                                                 78161
titulo                        terrenos en venta san luis
descripcion                                          NaN
tipodepropiedad                                      NaN
direccion                                            NaN
ciudad                                   San Luis Potosí
provincia                                San luis Potosí
antiguedad                                           NaN
habitaciones                                         NaN
garages                                              NaN
banos                                                NaN
metroscubiertos                                    78.00
metrostotales                                        NaN
idzona                                               NaN
lat                                                22.15
lng                                              -100.96
fecha                                2014-08-24 00:00:00
gimnasio                       

In [10]:
df_test["tipodepropiedad"].isna().sum()

7

In [11]:
df.columns[df.isna().any()].tolist()

['titulo',
 'descripcion',
 'tipodepropiedad',
 'direccion',
 'ciudad',
 'provincia',
 'antiguedad',
 'habitaciones',
 'garages',
 'banos',
 'metroscubiertos',
 'metrostotales',
 'idzona',
 'lat',
 'lng']

In [12]:
def getFeatureMean(tipodepropiedad, feature):
    propiedad = propiedades.loc[tipodepropiedad]
    value = propiedad[feature]
    return value

In [13]:
def esNulo(row, feature):
    return row[feature] == 0.0

def agregarNulls(row, feature):
    if(esNulo(row, feature)) and row["tipodepropiedad"] != "Vacio":
        value = propiedades.loc[row["tipodepropiedad"]][feature]
        return value
    else:
        return row[feature]

In [14]:
featuresToFix = ["habitaciones", "banos", "garages"]

df["tipodepropiedad"].fillna("Vacio", inplace=True)
df_test["tipodepropiedad"].fillna("Vacio", inplace=True)

df["provincia"].fillna("Vacio", inplace=True)
df_test["provincia"].fillna("Vacio", inplace=True)

df["ciudad"].fillna("Vacio", inplace=True)
df_test["ciudad"].fillna("Vacio", inplace=True)

df.fillna({'habitaciones': 0.0}, inplace = True)
df.fillna({'garages': 0.0}, inplace = True)
df.fillna({'banos': 0.0}, inplace = True)
df.fillna({'metroscubiertos': 0.0}, inplace = True)
df.fillna({'metrostotales': 0.0}, inplace = True)


df_test.fillna({'habitaciones': 0.0}, inplace = True)
df_test.fillna({'garages': 0.0}, inplace = True)
df_test.fillna({'banos': 0.0}, inplace = True)
df_test.fillna({'metroscubiertos': 0.0}, inplace = True)
df_test.fillna({'metrostotales': 0.0}, inplace = True)

In [15]:
df

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,nan,2.00,1.00,2.00,80.00,80.00,23533.00,nan,nan,2015-08-23,0.00,0.00,0.00,0.00,0.00,2273000.00
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.00,3.00,2.00,2.00,268.00,180.00,24514.00,19.31,-99.23,2013-06-28,0.00,0.00,0.00,1.00,1.00,3600000.00
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.00,3.00,2.00,2.00,144.00,166.00,48551.00,nan,nan,2015-10-17,0.00,0.00,0.00,0.00,0.00,1200000.00
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.00,2.00,1.00,1.00,63.00,67.00,53666.00,19.30,-99.69,2012-03-09,0.00,0.00,0.00,1.00,1.00,650000.00
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.00,2.00,1.00,1.00,95.00,95.00,47835.00,nan,nan,2016-06-07,0.00,0.00,0.00,0.00,0.00,1150000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,119879,bonita casas de 2 recamaras a 10 minutos del c...,vendo casa en bosques de ica residencial a 10 ...,Casa,BOSQUES,Zinacantepec,Edo. de México,0.00,2.00,2.00,1.00,67.00,0.00,53666.00,nan,nan,2015-02-08,0.00,0.00,0.00,0.00,0.00,650000.00
239996,259178,casa en condominio a 10 min. del centro de toluca,"casa con un jardin amplio, un cuarto de servic...",Casa,Filiberto Navas 325,Toluca,Edo. de México,0.00,3.00,3.00,3.00,200.00,250.00,51954.00,19.29,-99.69,2014-07-10,0.00,0.00,0.00,1.00,1.00,1940000.00
239997,131932,nicolas san juan,"departamento con excelente ubicación, muy cerc...",Apartamento,Nicolas San Juan,Benito Juárez,Distrito Federal,20.00,2.00,1.00,2.00,138.00,138.00,50003995.00,nan,nan,2015-03-03,0.00,0.00,0.00,0.00,0.00,3400000.00
239998,146867,casa sola. javier rojo gomez.,"casa sola, dividida en cuatro departamentos de...",Casa,Javier Rojo Gomez 120,Iztapalapa,Distrito Federal,20.00,4.00,0.00,4.00,235.00,137.00,24162.00,19.37,-99.08,2014-12-26,1.00,0.00,0.00,1.00,1.00,2890000.00


In [16]:
def esNuloYNoVacio(row, feature):
    return esNulo(row, feature) and row["tipodepropiedad"] != "Vacio"

for feature in featuresToFix:
    print ("Train : " + feature)
    df[feature] = df.apply(lambda row: agregarNulls(row, feature), axis = 1)
    print ("Test : " + feature)
    df_test[feature] = df_test.apply(lambda row: agregarNulls(row, feature), axis = 1)

Train : habitaciones
Test : habitaciones
Train : banos
Test : banos
Train : garages
Test : garages


In [17]:
df_test

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,4941,"casa en venta en miguel hidalgo, distrito federal",<p>excelente casa estilo moderno.</p>,Casa,Bosque de Cedros,Miguel Hidalgo,Distrito Federal,29.00,3.00,1.71,4.00,300.00,0.00,nan,19.41,-99.25,2013-07-20 00:00:00,0.00,0.00,0.00,0.00,0.00
1,51775,departamentos en venta en montebello,<p>departamento una recamara:\n</p><p>departam...,Apartamento,NaN,Mérida,Yucatán,nan,1.00,1.00,1.00,67.00,67.00,113851.00,21.03,-89.59,2015-10-24 00:00:00,0.00,0.00,0.00,0.00,0.00
2,115253,departamento nuevo delegación coyoacán de 87 m...,"departamento nuevo de 87.06 m2, 1 cajón de est...",Apartamento,"Pueblo de los Reyes, Coyoacán, Mexico D.F.",Coyoacán,Distrito Federal,0.00,2.00,1.00,2.00,87.00,100.00,23620.00,19.33,-99.15,2015-05-30 00:00:00,0.00,0.00,0.00,0.00,1.00
3,299321,departamento en venta en acapulco,<p> raíces dv001 precioso departamento tipo k...,Apartamento,NaN,Acapulco de Juárez,Guerrero,2.00,2.00,2.00,2.00,86.00,86.00,129347.00,16.86,-99.88,2015-04-02 00:00:00,0.00,0.00,0.00,0.00,0.00
4,173570,bonita casa sola equipada de dos niveles en lo...,"<p>casa sola, bonita de dos rec&aacute;maras u...",Casa,CEDROS,Tultitlán,Edo. de México,10.00,2.00,1.00,1.00,80.00,76.00,57125.00,19.64,-99.13,2013-08-15 00:00:00,0.00,0.00,0.00,1.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,75094,oportunidad!! se vende amplia casa en col. moc...,oportunidad!! ideal para oficina o casa habita...,Casa,Oriente 172 # 265,Venustiano Carranza,Distrito Federal,20.00,4.00,3.00,3.00,291.00,0.00,275741.00,19.43,-99.09,2015-09-19 00:00:00,0.00,0.00,0.00,1.00,1.00
59996,171847,colinas de ecatepec,"casa, sala comedor, patio de servicio, buenas ...",Casa,colinas,Ecatepec de Morelos,Edo. de México,10.00,3.00,1.00,2.00,71.00,87.00,57474.00,nan,nan,2016-10-19 00:00:00,0.00,0.00,0.00,1.00,1.00
59997,138313,estrene hermosa casa en sierra morena,hermosa casa lista para habitarse ubicada en f...,Casa,s/calle,Guadalupe,Nuevo León,5.00,3.00,2.00,2.00,102.00,0.00,72224.00,nan,nan,2014-12-02 00:00:00,0.00,0.00,0.00,1.00,1.00
59998,271268,zen house i venta de linda casa con acabados ...,hermosa casa con acabados de lujo en fracciona...,Casa,Zen House l,Querétaro,Querétaro,0.00,2.00,1.00,2.00,130.00,144.00,83960.00,20.59,-100.33,2016-10-21 00:00:00,0.00,0.00,0.00,1.00,1.00


In [18]:
df["metrostotales"] = df.apply(lambda row: asignarMetros(row['metroscubiertos'], row['metrostotales']),axis=1)
df_test["metrostotales"] = df.apply(lambda row: asignarMetros(row['metroscubiertos'], row['metrostotales']),axis=1)

In [19]:
meanPrecios = df.groupby(['provincia', "ciudad"])
meanPrecios = meanPrecios.agg({
         'precio': "mean"
    })


meanPrecios.head(100)

precio
provincia      ciudad                   
Aguascalientes Aguascalientes 1545951.02
               Asientos       1954869.60
               Calvillo       1653000.00
               Cosío           886666.67
               El Llano       1333333.33
...                                  ...
Colima         Colima         1598734.68
               Comala         3097800.00
               Coquimatlán     350000.00
               Cuauhtémoc     1284823.53
               Ixtlahuacán    1150000.00

[100 rows x 1 columns]

In [20]:
meanPreciosProvincia = df.groupby(['provincia'])
meanPreciosProvincia = meanPreciosProvincia.agg({
         'precio': "mean"
    })
meanPreciosProvincia.head(100)

,precio
provincia,
Aguascalientes,1477188.77
Baja California Norte,767441.75
Baja California Sur,1216452.82
Campeche,1570917.82
Chiapas,1924125.72
Chihuahua,1744530.61
Coahuila,1505305.45
Colima,1531380.16
Distrito Federal,3471705.09


In [21]:
meanPrecios.loc["Aguascalientes"].loc["Asientos"].precio

1954869.6

In [22]:
def getPrecioPromedio(row):
    try: 
        return meanPrecios.loc[row["provincia"]].loc[row["ciudad"]].precio
    except KeyError:
        return meanPreciosProvincia.loc[row["provincia"]].precio

Agrego precio promedio de la ciudad, para despues usarlo como un factor de ubicación.

In [23]:
def agregarPrecioPromedio(dfReceived):
    dfReceived["precio_promedio_ubicacion"] = dfReceived.apply(lambda row: getPrecioPromedio(row), axis = 1)
    return dfReceived

In [24]:
df = agregarPrecioPromedio(df)
df_test = agregarPrecioPromedio(df_test)

In [25]:
df

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,precio_promedio_ubicacion
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,nan,2.00,1.00,2.00,80.00,80.00,23533.00,nan,nan,2015-08-23,0.00,0.00,0.00,0.00,0.00,2273000.00,3384933.68
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.00,3.00,2.00,2.00,268.00,268.00,24514.00,19.31,-99.23,2013-06-28,0.00,0.00,0.00,1.00,1.00,3600000.00,4605868.67
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.00,3.00,2.00,2.00,144.00,166.00,48551.00,nan,nan,2015-10-17,0.00,0.00,0.00,0.00,0.00,1200000.00,869593.24
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.00,2.00,1.00,1.00,63.00,67.00,53666.00,19.30,-99.69,2012-03-09,0.00,0.00,0.00,1.00,1.00,650000.00,1472304.57
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.00,2.00,1.00,1.00,95.00,95.00,47835.00,nan,nan,2016-06-07,0.00,0.00,0.00,0.00,0.00,1150000.00,2749675.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,119879,bonita casas de 2 recamaras a 10 minutos del c...,vendo casa en bosques de ica residencial a 10 ...,Casa,BOSQUES,Zinacantepec,Edo. de México,0.00,2.00,2.00,1.00,67.00,67.00,53666.00,nan,nan,2015-02-08,0.00,0.00,0.00,0.00,0.00,650000.00,1472304.57
239996,259178,casa en condominio a 10 min. del centro de toluca,"casa con un jardin amplio, un cuarto de servic...",Casa,Filiberto Navas 325,Toluca,Edo. de México,0.00,3.00,3.00,3.00,200.00,250.00,51954.00,19.29,-99.69,2014-07-10,0.00,0.00,0.00,1.00,1.00,1940000.00,1449309.02
239997,131932,nicolas san juan,"departamento con excelente ubicación, muy cerc...",Apartamento,Nicolas San Juan,Benito Juárez,Distrito Federal,20.00,2.00,1.00,2.00,138.00,138.00,50003995.00,nan,nan,2015-03-03,0.00,0.00,0.00,0.00,0.00,3400000.00,3384933.68
239998,146867,casa sola. javier rojo gomez.,"casa sola, dividida en cuatro departamentos de...",Casa,Javier Rojo Gomez 120,Iztapalapa,Distrito Federal,20.00,4.00,1.71,4.00,235.00,235.00,24162.00,19.37,-99.08,2014-12-26,1.00,0.00,0.00,1.00,1.00,2890000.00,1699365.97


Agregando factor de ubicacion en base al precio promedio.

In [26]:
def agregarFactorUbicacion(dfReceived, max):
    dfReceived["factor_ubicacion"] = dfReceived.apply(lambda row: 100*(row["precio_promedio_ubicacion"]/max), axis=1)

In [27]:
max = df["precio_promedio_ubicacion"].max()
agregarFactorUbicacion(df, max)
agregarFactorUbicacion(df_test, max)

In [28]:
df.shape

(240000, 25)

Agrego año y mes (obtenidos de la fecha) y luego agrego el valor del dolar al momento de la venta.

In [29]:
df_train = agregarAnioMesYPrecioDelDolarALaFecha(df)
df_test = agregarAnioMesYPrecioDelDolarALaFecha(df_test)

In [30]:
dolar

,price,year,month
0,18.72,2018,9
1,19.09,2018,8
2,18.65,2018,7
3,19.92,2018,6
4,19.91,2018,5
...,...,...,...
79,12.85,2012,2
80,13.04,2012,1
81,13.95,2011,12
82,13.64,2011,11


In [31]:
def addColumns(dfReceived):
    dfReceived['metrostotales'].fillna(0, inplace=True)
    dfReceived['metrostotales'] = dfReceived.apply(lambda x: asignarMetros(x['metroscubiertos'], x['metrostotales']),axis=1)
    dfReceived["cant_ambientes"] = dfReceived["habitaciones"] + dfReceived["garages"] + dfReceived["banos"]
    dfReceived["metros_no_cubiertos"] = dfReceived["metrostotales"] - dfReceived["metroscubiertos"]
    dfReceived["precio_dolar_base_2012"] = dolar["price"].min()
    dfReceived["coeficiente_incremento_dolar"] = dfReceived["precio_dolar_base_2012"]/ dfReceived["precioDelDolar"]

In [32]:
addColumns(df_train)
addColumns(df_test)

df_train

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,precio_promedio_ubicacion,factor_ubicacion,year,month,precioDelDolar,cant_ambientes,metros_no_cubiertos,precio_dolar_base_2012,coeficiente_incremento_dolar
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,nan,2.00,1.00,2.00,80.00,80.00,23533.00,nan,nan,2015-08-23,0.00,0.00,0.00,0.00,0.00,2273000.00,3384933.68,27.08,2015,8,16.75,5.00,0.00,12.13,0.72
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.00,3.00,2.00,2.00,268.00,268.00,24514.00,19.31,-99.23,2013-06-28,0.00,0.00,0.00,1.00,1.00,3600000.00,4605868.67,36.85,2013,6,12.95,7.00,0.00,12.13,0.94
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.00,3.00,2.00,2.00,144.00,166.00,48551.00,nan,nan,2015-10-17,0.00,0.00,0.00,0.00,0.00,1200000.00,869593.24,6.96,2015,10,16.50,7.00,22.00,12.13,0.74
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.00,2.00,1.00,1.00,63.00,67.00,53666.00,19.30,-99.69,2012-03-09,0.00,0.00,0.00,1.00,1.00,650000.00,1472304.57,11.78,2012,3,12.81,4.00,4.00,12.13,0.95
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.00,2.00,1.00,1.00,95.00,95.00,47835.00,nan,nan,2016-06-07,0.00,0.00,0.00,0.00,0.00,1150000.00,2749675.37,22.00,2016,6,18.28,4.00,0.00,12.13,0.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,119879,bonita casas de 2 recamaras a 10 minutos del c...,vendo casa en bosques de ica residencial a 10 ...,Casa,BOSQUES,Zinacantepec,Edo. de México,0.00,2.00,2.00,1.00,67.00,67.00,53666.00,nan,nan,2015-02-08,0.00,0.00,0.00,0.00,0.00,650000.00,1472304.57,11.78,2015,2,14.95,5.00,0.00,12.13,0.81
239996,259178,casa en condominio a 10 min. del centro de toluca,"casa con un jardin amplio, un cuarto de servic...",Casa,Filiberto Navas 325,Toluca,Edo. de México,0.00,3.00,3.00,3.00,200.00,250.00,51954.00,19.29,-99.69,2014-07-10,0.00,0.00,0.00,1.00,1.00,1940000.00,1449309.02,11.59,2014,7,13.22,9.00,50.00,12.13,0.92
239997,131932,nicolas san juan,"departamento con excelente ubicación, muy cerc...",Apartamento,Nicolas San Juan,Benito Juárez,Distrito Federal,20.00,2.00,1.00,2.00,138.00,138.00,50003995.00,nan,nan,2015-03-03,0.00,0.00,0.00,0.00,0.00,3400000.00,3384933.68,27.08,2015,3,15.26,5.00,0.00,12.13,0.80
239998,146867,casa sola. javier rojo gomez.,"casa sola, dividida en cuatro departamentos de...",Casa,Javier Rojo Gomez 120,Iztapalapa,Distrito Federal,20.00,4.00,1.71,4.00,235.00,235.00,24162.00,19.37,-99.08,2014-12-26,1.00,0.00,0.00,1.00,1.00,2890000.00,1699365.97,13.59,2014,12,14.75,9.71,0.00,12.13,0.82


In [33]:
def addFeature(listaPalabras, nombreDelFeature):
    df_train[nombreDelFeature] = 0
    df_test[nombreDelFeature] = 0
    for palabra in listaPalabras:
        df_train.loc[df_train['descripcion'].str.contains(palabra)==True, nombreDelFeature] = 1
        df_test.loc[df_test['descripcion'].str.contains(palabra)==True, nombreDelFeature] = 1
    print("Feature (train) " + nombreDelFeature + ":" + str(df_train[nombreDelFeature].sum()))
    print("Feature (test) " + nombreDelFeature + ":" + str(df_test[nombreDelFeature].sum()))

In [34]:
addFeature(["luminoso"], "es_luminoso")
addFeature(["amplio"], "es_amplio")
addFeature(["balcon"], "tiene_balcon")
addFeature(["nuevo"], "es_nuevo")
addFeature(["buen estado", "excelente estado", "excelentes condiciones"], "buen_estado")
addFeature(["excelente ubicacion", "buena ubicaion", "ubicacion"], "excelente_ubicacion")
addFeature(["privada", "barrio privado", "privado"], "es_privado")
addFeature(["seguridad", "seguro", "vigilancia", "policía", "policia"], "es_seguro")

Feature (train) es_luminoso:381
Feature (test) es_luminoso:94
Feature (train) es_amplio:37306
Feature (test) es_amplio:9325
Feature (train) tiene_balcon:4839
Feature (test) tiene_balcon:1232
Feature (train) es_nuevo:9775
Feature (test) es_nuevo:2434
Feature (train) buen_estado:10026
Feature (test) buen_estado:2545
Feature (train) excelente_ubicacion:8734
Feature (test) excelente_ubicacion:2111
Feature (train) es_privado:29178
Feature (test) es_privado:7273
Feature (train) es_seguro:53907
Feature (test) es_seguro:13569


In [35]:
def sumarFeatures(dfReceived):
    dfReceived["cant_features"] = dfReceived["es_luminoso"] + dfReceived["es_amplio"] +dfReceived["tiene_balcon"] + \
                                  dfReceived["es_nuevo"] + dfReceived["buen_estado"] + dfReceived["excelente_ubicacion"] + \
                                  dfReceived["es_privado"] +dfReceived["es_seguro"]

In [36]:
sumarFeatures(df_train)
sumarFeatures(df_test)

In [37]:
df_test

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio_promedio_ubicacion,factor_ubicacion,year,month,precioDelDolar,cant_ambientes,metros_no_cubiertos,precio_dolar_base_2012,coeficiente_incremento_dolar,es_luminoso,es_amplio,tiene_balcon,es_nuevo,buen_estado,excelente_ubicacion,es_privado,es_seguro,cant_features
0,4941,"casa en venta en miguel hidalgo, distrito federal",<p>excelente casa estilo moderno.</p>,Casa,Bosque de Cedros,Miguel Hidalgo,Distrito Federal,29.00,3.00,1.71,4.00,300.00,300.00,nan,19.41,-99.25,2013-07-20,0.00,0.00,0.00,0.00,0.00,4496525.64,35.97,2013,7,12.73,8.71,0.00,12.13,0.95,0,0,0,0,0,0,0,0,0
1,51775,departamentos en venta en montebello,<p>departamento una recamara:\n</p><p>departam...,Apartamento,NaN,Mérida,Yucatán,nan,1.00,1.00,1.00,67.00,268.00,113851.00,21.03,-89.59,2015-10-24,0.00,0.00,0.00,0.00,0.00,2028803.22,16.23,2015,10,16.50,3.00,201.00,12.13,0.74,0,0,0,0,0,0,0,1,1
2,115253,departamento nuevo delegación coyoacán de 87 m...,"departamento nuevo de 87.06 m2, 1 cajón de est...",Apartamento,"Pueblo de los Reyes, Coyoacán, Mexico D.F.",Coyoacán,Distrito Federal,0.00,2.00,1.00,2.00,87.00,166.00,23620.00,19.33,-99.15,2015-05-30,0.00,0.00,0.00,0.00,1.00,3407009.15,27.26,2015,5,15.38,5.00,79.00,12.13,0.79,0,0,0,1,0,0,0,1,2
3,299321,departamento en venta en acapulco,<p> raíces dv001 precioso departamento tipo k...,Apartamento,NaN,Acapulco de Juárez,Guerrero,2.00,2.00,2.00,2.00,86.00,86.00,129347.00,16.86,-99.88,2015-04-02,0.00,0.00,0.00,0.00,0.00,2585813.16,20.69,2015,4,15.35,6.00,0.00,12.13,0.79,0,0,0,0,0,0,0,1,1
4,173570,bonita casa sola equipada de dos niveles en lo...,"<p>casa sola, bonita de dos rec&aacute;maras u...",Casa,CEDROS,Tultitlán,Edo. de México,10.00,2.00,1.00,1.00,80.00,95.00,57125.00,19.64,-99.13,2013-08-15,0.00,0.00,0.00,1.00,1.00,914738.25,7.32,2013,8,13.38,4.00,15.00,12.13,0.91,0,0,0,0,1,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,75094,oportunidad!! se vende amplia casa en col. moc...,oportunidad!! ideal para oficina o casa habita...,Casa,Oriente 172 # 265,Venustiano Carranza,Distrito Federal,20.00,4.00,3.00,3.00,291.00,291.00,275741.00,19.43,-99.09,2015-09-19,0.00,0.00,0.00,1.00,1.00,1943750.82,15.55,2015,9,16.92,10.00,0.00,12.13,0.72,0,0,0,0,0,0,0,0,0
59996,171847,colinas de ecatepec,"casa, sala comedor, patio de servicio, buenas ...",Casa,colinas,Ecatepec de Morelos,Edo. de México,10.00,3.00,1.00,2.00,71.00,71.00,57474.00,nan,nan,2016-10-19,0.00,0.00,0.00,1.00,1.00,1096164.52,8.77,2016,10,18.86,6.00,0.00,12.13,0.64,0,0,0,0,0,0,0,0,0
59997,138313,estrene hermosa casa en sierra morena,hermosa casa lista para habitarse ubicada en f...,Casa,s/calle,Guadalupe,Nuevo León,5.00,3.00,2.00,2.00,102.00,102.00,72224.00,nan,nan,2014-12-02,0.00,0.00,0.00,1.00,1.00,1475068.93,11.80,2014,12,14.75,7.00,0.00,12.13,0.82,0,1,0,0,0,0,1,1,3
59998,271268,zen house i venta de linda casa con acabados ...,hermosa casa con acabados de lujo en fracciona...,Casa,Zen House l,Querétaro,Querétaro,0.00,2.00,1.00,2.00,130.00,130.00,83960.00,20.59,-100.33,2016-10-21,0.00,0.00,0.00,1.00,1.00,2271790.41,18.17,2016,10,18.86,5.00,0.00,12.13,0.64,0,0,0,0,0,0,0,1,1


In [38]:
# Droppeamos strings y columnas complejas y repetidas
drop_cols = ['titulo', 'descripcion', 'direccion', 'lat', 'lng', 'fecha', 'ciudad', 'provincia']
df_train = df_train.drop(drop_cols, axis=1).copy()

df_test = df_test.drop(drop_cols, axis=1).copy()
print(f"Columnas ({len(df_train.columns)}): {df_train.columns.tolist()}")
df_train.head()

Columnas (33): ['id', 'tipodepropiedad', 'antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'idzona', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio', 'precio_promedio_ubicacion', 'factor_ubicacion', 'year', 'month', 'precioDelDolar', 'cant_ambientes', 'metros_no_cubiertos', 'precio_dolar_base_2012', 'coeficiente_incremento_dolar', 'es_luminoso', 'es_amplio', 'tiene_balcon', 'es_nuevo', 'buen_estado', 'excelente_ubicacion', 'es_privado', 'es_seguro', 'cant_features']


,id,tipodepropiedad,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,precio_promedio_ubicacion,factor_ubicacion,year,month,precioDelDolar,cant_ambientes,metros_no_cubiertos,precio_dolar_base_2012,coeficiente_incremento_dolar,es_luminoso,es_amplio,tiene_balcon,es_nuevo,buen_estado,excelente_ubicacion,es_privado,es_seguro,cant_features
0,254099,Apartamento,nan,2.00,1.00,2.00,80.00,80.00,23533.00,0.00,0.00,0.00,0.00,0.00,2273000.00,3384933.68,27.08,2015,8,16.75,5.00,0.00,12.13,0.72,0,0,0,0,0,0,0,0,0
1,53461,Casa en condominio,10.00,3.00,2.00,2.00,268.00,268.00,24514.00,0.00,0.00,0.00,1.00,1.00,3600000.00,4605868.67,36.85,2013,6,12.95,7.00,0.00,12.13,0.94,0,0,0,0,1,0,0,0,1
2,247984,Casa,5.00,3.00,2.00,2.00,144.00,166.00,48551.00,0.00,0.00,0.00,0.00,0.00,1200000.00,869593.24,6.96,2015,10,16.50,7.00,22.00,12.13,0.74,0,1,0,0,0,1,1,1,4
3,209067,Casa,1.00,2.00,1.00,1.00,63.00,67.00,53666.00,0.00,0.00,0.00,1.00,1.00,650000.00,1472304.57,11.78,2012,3,12.81,4.00,4.00,12.13,0.95,0,0,0,0,0,0,1,1,2
4,185997,Apartamento,10.00,2.00,1.00,1.00,95.00,95.00,47835.00,0.00,0.00,0.00,0.00,0.00,1150000.00,2749675.37,22.00,2016,6,18.28,4.00,0.00,12.13,0.66,0,0,0,0,1,1,0,0,2


In [39]:
df_test.dtypes

id                                int64
tipodepropiedad                  object
antiguedad                      float64
habitaciones                    float64
garages                         float64
banos                           float64
metroscubiertos                 float64
metrostotales                   float64
idzona                          float64
gimnasio                        float64
usosmultiples                   float64
piscina                         float64
escuelascercanas                float64
centroscomercialescercanos      float64
precio_promedio_ubicacion       float64
factor_ubicacion                float64
year                              int64
month                             int64
precioDelDolar                  float64
cant_ambientes                  float64
metros_no_cubiertos             float64
precio_dolar_base_2012          float64
coeficiente_incremento_dolar    float64
es_luminoso                       int64
es_amplio                         int64


In [40]:
display(df_train.isnull().sum())
numeric_columns_with_nulls = list(set(df_train.columns[df_train.isnull().sum() > 0].tolist()) - 
                                  set(['tipodepropiedad', 'ciudad', 'provincia']))
print(numeric_columns_with_nulls)

id                                  0
tipodepropiedad                     0
antiguedad                      43555
habitaciones                        0
garages                             0
banos                               0
metroscubiertos                     0
metrostotales                       0
idzona                          28621
gimnasio                            0
usosmultiples                       0
piscina                             0
escuelascercanas                    0
centroscomercialescercanos          0
precio                              0
precio_promedio_ubicacion           0
factor_ubicacion                    0
year                                0
month                               0
precioDelDolar                      0
cant_ambientes                      0
metros_no_cubiertos                 0
precio_dolar_base_2012              0
coeficiente_incremento_dolar        0
es_luminoso                         0
es_amplio                           0
tiene_balcon

['antiguedad', 'idzona']


In [41]:
#ciudades_en_train = df_train.ciudad.unique()
#ciudades_en_test = df_test.ciudad.unique()

tipodepropiedad_en_train = df_train.tipodepropiedad.unique()
tipodepropiedad_en_test = df_test.tipodepropiedad.unique()

#provincia_en_train = df_train.provincia.unique()
#provincia_en_test = df_test.provincia.unique()

In [42]:
# Para los nulls numéricos, usar un Imputer con strategy mean (reemplazamos los NaN por el promedio)
# Para no leakear, spliteamos el dataset antes
df_train_ohe = pd.get_dummies(df_train, dummy_na=True)
print("First")
df_test_ohe = pd.get_dummies(df_test, dummy_na=True)

First


In [43]:
X = df_train_ohe.drop("precio", axis=1)
y = df_train_ohe['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

display(df_train_ohe)

,id,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,precio_promedio_ubicacion,factor_ubicacion,year,month,precioDelDolar,cant_ambientes,metros_no_cubiertos,precio_dolar_base_2012,coeficiente_incremento_dolar,es_luminoso,es_amplio,tiene_balcon,es_nuevo,buen_estado,excelente_ubicacion,es_privado,es_seguro,cant_features,tipodepropiedad_Apartamento,tipodepropiedad_Bodega comercial,tipodepropiedad_Casa,tipodepropiedad_Casa en condominio,tipodepropiedad_Casa uso de suelo,tipodepropiedad_Departamento Compartido,tipodepropiedad_Duplex,tipodepropiedad_Edificio,tipodepropiedad_Garage,tipodepropiedad_Hospedaje,tipodepropiedad_Huerta,tipodepropiedad_Inmuebles productivos urbanos,tipodepropiedad_Local Comercial,tipodepropiedad_Local en centro comercial,tipodepropiedad_Lote,tipodepropiedad_Nave industrial,tipodepropiedad_Oficina comercial,tipodepropiedad_Otros,tipodepropiedad_Quinta Vacacional,tipodepropiedad_Rancho,tipodepropiedad_Terreno,tipodepropiedad_Terreno comercial,tipodepropiedad_Terreno industrial,tipodepropiedad_Vacio,tipodepropiedad_Villa,tipodepropiedad_nan
0,254099,nan,2.00,1.00,2.00,80.00,80.00,23533.00,0.00,0.00,0.00,0.00,0.00,2273000.00,3384933.68,27.08,2015,8,16.75,5.00,0.00,12.13,0.72,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,53461,10.00,3.00,2.00,2.00,268.00,268.00,24514.00,0.00,0.00,0.00,1.00,1.00,3600000.00,4605868.67,36.85,2013,6,12.95,7.00,0.00,12.13,0.94,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,247984,5.00,3.00,2.00,2.00,144.00,166.00,48551.00,0.00,0.00,0.00,0.00,0.00,1200000.00,869593.24,6.96,2015,10,16.50,7.00,22.00,12.13,0.74,0,1,0,0,0,1,1,1,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,209067,1.00,2.00,1.00,1.00,63.00,67.00,53666.00,0.00,0.00,0.00,1.00,1.00,650000.00,1472304.57,11.78,2012,3,12.81,4.00,4.00,12.13,0.95,0,0,0,0,0,0,1,1,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,185997,10.00,2.00,1.00,1.00,95.00,95.00,47835.00,0.00,0.00,0.00,0.00,0.00,1150000.00,2749675.37,22.00,2016,6,18.28,4.00,0.00,12.13,0.66,0,0,0,0,1,1,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,119879,0.00,2.00,2.00,1.00,67.00,67.00,53666.00,0.00,0.00,0.00,0.00,0.00,650000.00,1472304.57,11.78,2015,2,14.95,5.00,0.00,12.13,0.81,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
239996,259178,0.00,3.00,3.00,3.00,200.00,250.00,51954.00,0.00,0.00,0.00,1.00,1.00,1940000.00,1449309.02,11.59,2014,7,13.22,9.00,50.00,12.13,0.92,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
239997,131932,20.00,2.00,1.00,2.00,138.00,138.00,50003995.00,0.00,0.00,0.00,0.00,0.00,3400000.00,3384933.68,27.08,2015,3,15.26,5.00,0.00,12.13,0.80,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
239998,146867,20.00,4.00,1.71,4.00,235.00,235.00,24162.00,1.00,0.00,0.00,1.00,1.00,2890000.00,1699365.97,13.59,2014,12,14.75,9.71,0.00,12.13,0.82,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [44]:
# Con dummy_na=True, creamos la categoria "Es nulo" como una coordenada más de los one-hot vectors
# Comentar: ¿Hay leaks acá? ¿Sí / No? ¿Por qué?
#df2 = pd.get_dummies(df2, dummy_na=True)
print(f"Cantidad de columnas después del one-hot-encoding: {len(df_train_ohe.columns)}")
df_train_ohe.head(10)

Cantidad de columnas después del one-hot-encoding: 58


,id,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,precio_promedio_ubicacion,factor_ubicacion,year,month,precioDelDolar,cant_ambientes,metros_no_cubiertos,precio_dolar_base_2012,coeficiente_incremento_dolar,es_luminoso,es_amplio,tiene_balcon,es_nuevo,buen_estado,excelente_ubicacion,es_privado,es_seguro,cant_features,tipodepropiedad_Apartamento,tipodepropiedad_Bodega comercial,tipodepropiedad_Casa,tipodepropiedad_Casa en condominio,tipodepropiedad_Casa uso de suelo,tipodepropiedad_Departamento Compartido,tipodepropiedad_Duplex,tipodepropiedad_Edificio,tipodepropiedad_Garage,tipodepropiedad_Hospedaje,tipodepropiedad_Huerta,tipodepropiedad_Inmuebles productivos urbanos,tipodepropiedad_Local Comercial,tipodepropiedad_Local en centro comercial,tipodepropiedad_Lote,tipodepropiedad_Nave industrial,tipodepropiedad_Oficina comercial,tipodepropiedad_Otros,tipodepropiedad_Quinta Vacacional,tipodepropiedad_Rancho,tipodepropiedad_Terreno,tipodepropiedad_Terreno comercial,tipodepropiedad_Terreno industrial,tipodepropiedad_Vacio,tipodepropiedad_Villa,tipodepropiedad_nan
0,254099,nan,2.00,1.00,2.00,80.00,80.00,23533.00,0.00,0.00,0.00,0.00,0.00,2273000.00,3384933.68,27.08,2015,8,16.75,5.00,0.00,12.13,0.72,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,53461,10.00,3.00,2.00,2.00,268.00,268.00,24514.00,0.00,0.00,0.00,1.00,1.00,3600000.00,4605868.67,36.85,2013,6,12.95,7.00,0.00,12.13,0.94,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,247984,5.00,3.00,2.00,2.00,144.00,166.00,48551.00,0.00,0.00,0.00,0.00,0.00,1200000.00,869593.24,6.96,2015,10,16.50,7.00,22.00,12.13,0.74,0,1,0,0,0,1,1,1,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,209067,1.00,2.00,1.00,1.00,63.00,67.00,53666.00,0.00,0.00,0.00,1.00,1.00,650000.00,1472304.57,11.78,2012,3,12.81,4.00,4.00,12.13,0.95,0,0,0,0,0,0,1,1,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,185997,10.00,2.00,1.00,1.00,95.00,95.00,47835.00,0.00,0.00,0.00,0.00,0.00,1150000.00,2749675.37,22.00,2016,6,18.28,4.00,0.00,12.13,0.66,0,0,0,0,1,1,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,126147,5.00,2.00,1.00,1.00,75.00,90.00,23650.00,0.00,0.00,0.00,0.00,1.00,1100000.00,3407009.15,27.26,2014,3,13.06,4.00,15.00,12.13,0.93,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,139233,nan,3.00,1.00,2.00,140.00,160.00,73510.00,0.00,0.00,0.00,0.00,0.00,1150000.00,1971185.86,15.77,2016,2,18.14,6.00,20.00,12.13,0.67,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,5013,2.00,4.00,2.00,3.00,293.00,293.00,130510.00,0.00,0.00,0.00,0.00,0.00,4200000.00,2215471.73,17.72,2016,10,18.86,9.00,0.00,12.13,0.64,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,44962,1.00,2.00,1.00,1.00,58.00,58.00,9010.00,0.00,0.00,0.00,1.00,1.00,310000.00,828058.13,6.62,2014,1,13.36,4.00,0.00,12.13,0.91,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,134537,nan,2.69,0.05,1.69,250.00,250.00,59171.00,0.00,0.00,0.00,0.00,0.00,6200000.00,832866.37,6.66,2016,12,20.73,4.43,0.00,12.13,0.59,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [45]:
from sklearn.impute import SimpleImputer
for c in numeric_columns_with_nulls:
    imp = SimpleImputer()
    X_train[c] = imp.fit_transform(X_train[[c]])
    X[c] = imp.fit_transform(X[[c]])
    X_test[c] = imp.transform(X_test[[c]])
    df_test_ohe[c] = imp.transform(df_test_ohe[[c]])
    
print ("Finished")

Finished


Inserto modelos

In [46]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [58]:
#Score con min_samples_split: 4  y min samples leaf:5 es: 0.7810914578619199
#Score con min_samples_split: 3  y min samples leaf:5 es: 0.7813268576824561
#Score con min_samples_split: 3  y min samples leaf:4 es: 0.7829347132494933
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators = 400, min_samples_leaf = 2, 
                        min_samples_split = 3, max_depth = 20, 
                        n_jobs = -1)
rfr.fit(X_train, y_train)
print("Score con " + str(item) + " es: " + str(rfr.score(X_test, y_test)))

Score con 3 es: 0.7851511249408217


**BLENDING MODEL**

In [59]:
listaTrain = set(df_train_ohe.columns.values.tolist())
listaTest = set(df_test_ohe.columns.values.tolist())
res = listaTrain - listaTest
res

{'precio', 'tipodepropiedad_Garage', 'tipodepropiedad_Hospedaje'}

In [60]:
#for ciudad in ciudades_en_train:
#    if ciudad not in ciudades_en_test:
#        df_test_ohe['ciudad_'+ str(ciudad)] = 0
        
#for provincia in provincia_en_train:
#    if provincia not in provincia_en_test:
#        df_test_ohe['provincia_'+ str(provincia)] = 0
        
for tipodepropiedad in tipodepropiedad_en_train:
    if tipodepropiedad not in tipodepropiedad_en_test:
        df_test_ohe['tipodepropiedad_'+ str(tipodepropiedad)] = 0

In [61]:
listaTrain = set(df_train_ohe.columns.values.tolist())
listaTest = set(df_test_ohe.columns.values.tolist())
res = listaTrain - listaTest
res

{'precio'}

In [ ]:
df_test_ohe

In [62]:
df_test_ohe.rename(columns={"price_x": "precioDelDolar"}, inplace = True)
df_test_ohe

,id,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio_promedio_ubicacion,factor_ubicacion,year,month,precioDelDolar,cant_ambientes,metros_no_cubiertos,precio_dolar_base_2012,coeficiente_incremento_dolar,es_luminoso,es_amplio,tiene_balcon,es_nuevo,buen_estado,excelente_ubicacion,es_privado,es_seguro,cant_features,tipodepropiedad_Apartamento,tipodepropiedad_Bodega comercial,tipodepropiedad_Casa,tipodepropiedad_Casa en condominio,tipodepropiedad_Casa uso de suelo,tipodepropiedad_Departamento Compartido,tipodepropiedad_Duplex,tipodepropiedad_Edificio,tipodepropiedad_Huerta,tipodepropiedad_Inmuebles productivos urbanos,tipodepropiedad_Local Comercial,tipodepropiedad_Local en centro comercial,tipodepropiedad_Lote,tipodepropiedad_Nave industrial,tipodepropiedad_Oficina comercial,tipodepropiedad_Otros,tipodepropiedad_Quinta Vacacional,tipodepropiedad_Rancho,tipodepropiedad_Terreno,tipodepropiedad_Terreno comercial,tipodepropiedad_Terreno industrial,tipodepropiedad_Vacio,tipodepropiedad_Villa,tipodepropiedad_nan,tipodepropiedad_Hospedaje,tipodepropiedad_Garage
0,4941,29.00,3.00,1.71,4.00,300.00,300.00,2423467.82,0.00,0.00,0.00,0.00,0.00,4496525.64,35.97,2013,7,12.73,8.71,0.00,12.13,0.95,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,51775,8.12,1.00,1.00,1.00,67.00,268.00,113851.00,0.00,0.00,0.00,0.00,0.00,2028803.22,16.23,2015,10,16.50,3.00,201.00,12.13,0.74,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,115253,0.00,2.00,1.00,2.00,87.00,166.00,23620.00,0.00,0.00,0.00,0.00,1.00,3407009.15,27.26,2015,5,15.38,5.00,79.00,12.13,0.79,0,0,0,1,0,0,0,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,299321,2.00,2.00,2.00,2.00,86.00,86.00,129347.00,0.00,0.00,0.00,0.00,0.00,2585813.16,20.69,2015,4,15.35,6.00,0.00,12.13,0.79,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,173570,10.00,2.00,1.00,1.00,80.00,95.00,57125.00,0.00,0.00,0.00,1.00,1.00,914738.25,7.32,2013,8,13.38,4.00,15.00,12.13,0.91,0,0,0,0,1,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,75094,20.00,4.00,3.00,3.00,291.00,291.00,275741.00,0.00,0.00,0.00,1.00,1.00,1943750.82,15.55,2015,9,16.92,10.00,0.00,12.13,0.72,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59996,171847,10.00,3.00,1.00,2.00,71.00,71.00,57474.00,0.00,0.00,0.00,1.00,1.00,1096164.52,8.77,2016,10,18.86,6.00,0.00,12.13,0.64,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59997,138313,5.00,3.00,2.00,2.00,102.00,102.00,72224.00,0.00,0.00,0.00,1.00,1.00,1475068.93,11.80,2014,12,14.75,7.00,0.00,12.13,0.82,0,1,0,0,0,0,1,1,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59998,271268,0.00,2.00,1.00,2.00,130.00,130.00,83960.00,0.00,0.00,0.00,1.00,1.00,2271790.41,18.17,2016,10,18.86,5.00,0.00,12.13,0.64,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [65]:
y_test

132827   2300000.00
210008   2262945.00
189056   3800000.00
131466   1000000.00
207302    376200.00
            ...    
187516   2150000.00
176440   3300000.00
105368   9980000.00
48757     565000.00
83779    1910000.00
Name: precio, Length: 60000, dtype: float64

In [67]:
y_false_pred = rfr.predict(X_test)

In [68]:
y_false_pred

array([1559686.11355839, 2171198.25546382, 3429001.66890207, ...,
       6804511.02250392,  572647.16183247, 2801744.66503543])

In [70]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_test, y_false_pred))

589229.7924608523


In [71]:
df_test_ohe = df_test_ohe.loc[:, ~df_test_ohe.columns.duplicated()]
df_test_ohe

,id,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio_promedio_ubicacion,factor_ubicacion,year,month,precioDelDolar,cant_ambientes,metros_no_cubiertos,precio_dolar_base_2012,coeficiente_incremento_dolar,es_luminoso,es_amplio,tiene_balcon,es_nuevo,buen_estado,excelente_ubicacion,es_privado,es_seguro,cant_features,tipodepropiedad_Apartamento,tipodepropiedad_Bodega comercial,tipodepropiedad_Casa,tipodepropiedad_Casa en condominio,tipodepropiedad_Casa uso de suelo,tipodepropiedad_Departamento Compartido,tipodepropiedad_Duplex,tipodepropiedad_Edificio,tipodepropiedad_Huerta,tipodepropiedad_Inmuebles productivos urbanos,tipodepropiedad_Local Comercial,tipodepropiedad_Local en centro comercial,tipodepropiedad_Lote,tipodepropiedad_Nave industrial,tipodepropiedad_Oficina comercial,tipodepropiedad_Otros,tipodepropiedad_Quinta Vacacional,tipodepropiedad_Rancho,tipodepropiedad_Terreno,tipodepropiedad_Terreno comercial,tipodepropiedad_Terreno industrial,tipodepropiedad_Vacio,tipodepropiedad_Villa,tipodepropiedad_nan,tipodepropiedad_Hospedaje,tipodepropiedad_Garage
0,4941,29.00,3.00,1.71,4.00,300.00,300.00,2423467.82,0.00,0.00,0.00,0.00,0.00,4496525.64,35.97,2013,7,12.73,8.71,0.00,12.13,0.95,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,51775,8.12,1.00,1.00,1.00,67.00,268.00,113851.00,0.00,0.00,0.00,0.00,0.00,2028803.22,16.23,2015,10,16.50,3.00,201.00,12.13,0.74,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,115253,0.00,2.00,1.00,2.00,87.00,166.00,23620.00,0.00,0.00,0.00,0.00,1.00,3407009.15,27.26,2015,5,15.38,5.00,79.00,12.13,0.79,0,0,0,1,0,0,0,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,299321,2.00,2.00,2.00,2.00,86.00,86.00,129347.00,0.00,0.00,0.00,0.00,0.00,2585813.16,20.69,2015,4,15.35,6.00,0.00,12.13,0.79,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,173570,10.00,2.00,1.00,1.00,80.00,95.00,57125.00,0.00,0.00,0.00,1.00,1.00,914738.25,7.32,2013,8,13.38,4.00,15.00,12.13,0.91,0,0,0,0,1,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,75094,20.00,4.00,3.00,3.00,291.00,291.00,275741.00,0.00,0.00,0.00,1.00,1.00,1943750.82,15.55,2015,9,16.92,10.00,0.00,12.13,0.72,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59996,171847,10.00,3.00,1.00,2.00,71.00,71.00,57474.00,0.00,0.00,0.00,1.00,1.00,1096164.52,8.77,2016,10,18.86,6.00,0.00,12.13,0.64,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59997,138313,5.00,3.00,2.00,2.00,102.00,102.00,72224.00,0.00,0.00,0.00,1.00,1.00,1475068.93,11.80,2014,12,14.75,7.00,0.00,12.13,0.82,0,1,0,0,0,0,1,1,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59998,271268,0.00,2.00,1.00,2.00,130.00,130.00,83960.00,0.00,0.00,0.00,1.00,1.00,2271790.41,18.17,2016,10,18.86,5.00,0.00,12.13,0.64,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_test_ohe

In [72]:
#Cosas para entregar
X_real_test = df_test_ohe
y_real_pred = rfr.predict(X_real_test)
y_real_pred

array([8118461.64569445, 8400717.94186508, 9163645.050625  , ...,
       7997815.73535083, 8781137.07086585, 8848428.4983135 ])

In [73]:
res = pd.DataFrame(y_real_pred, index= df_test_ohe.index, columns=['precio'])
res2 = res
res2['id'] = df_test_ohe["id"]
res = res.reset_index(drop=True)
res.set_index('id', inplace = True)
res.rename(columns={"precio": "target"}, inplace = True)

In [74]:
res

,target
id,
4941,8118461.65
51775,8400717.94
115253,9163645.05
299321,8289042.73
173570,8234482.67
...,...
75094,8084409.74
171847,8047541.61
138313,7997815.74


In [ ]:
res.to_csv("randomForest.csv", header=True)